In [1]:
import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go

df = pd.read_csv("/kaggle/input/na-lcs-summer-2020-player-proximities/proximities.csv")
df.drop("Unnamed: 0", inplace=True, axis=1)
teams = df['game_won'].drop_duplicates().tolist()

# Comparing NA LCS Support Proximities

## Introduction

One important statistic in professional League of Legends is proximity. That is to say, how often one player is near one of their teammates. This can be useful to figure out the playstyle of certain teams. If your opposing team's *Support* player doesn't play around their *Midlaner* a lot, that's a pattern that may be abused. Similarly, if their *Jungler* tends to play around his *Toplaner*, then it may be a good idea to provide extra support for your own *Toplaner*. In this notebook I'll try to compare the Supports of the North American League of Legends Championship Series (NA LCS) and attempt to graph the patterns in their play through proximities

## Proximity data

The data was gathered using [this](https://github.com/Steeeephen/loltracker) program, which tracks the locations of players over time from Youtube videos. To get the proximity stat, the distance between players is calculated at every second and the proportion of time spent within 50 units of each other is returned. This proportion then denotes how often any two players are in 'close proximity', with higher values meaning more time spent in each other's vicinity.

The data was gathered using a subset of 75 games from the Summer split of the NA LCS. 


In [2]:
df.head()

,__video_title,side,team,opposition,game_won,player_role,teammate_role,player_champion,teammate_champion,proximity
0,100 Thieves vs Cloud9 Full - LCS Summer 2020 W...,blue,100t,c9,100t,support,top,karma,camille,4.166667
1,100 Thieves vs Cloud9 Full - LCS Summer 2020 W...,blue,100t,c9,100t,support,jgl,karma,olaf,35.700758
2,100 Thieves vs Cloud9 Full - LCS Summer 2020 W...,blue,100t,c9,100t,support,mid,karma,galio,25.189394
3,100 Thieves vs Cloud9 Full - LCS Summer 2020 W...,blue,100t,c9,100t,support,adc,karma,ashe,82.386364
4,100 Thieves vs Cloud9 Full - LCS Summer 2020 W...,blue,100t,c9,100t,jungle,top,olaf,camille,30.871212


# Method

First I'll take all the entries for Support players. The dataset includes entries for Junglers too, but that's not needed for this

In [3]:
df_sup = df[df['player_role']=="support"]

Next I'll calculate the mean proximities between the Support and their teammates for every team. Seeing as there will be different value ranges for each role, I'll normalise them to fall between 0 and 1

In [4]:
df3 = pd.DataFrame(index=teams)

for role in ['top', 'jgl', 'mid', 'adc']:
    df3[role] = df_sup[df_sup['teammate_role']==role].groupby('team').mean()['proximity'].tolist()

    
for role in ['top','jgl','mid','adc']:
    df3[role] -= df3[role].min()
    df3[role] /= df3[role].max()
df3

,top,jgl,mid,adc
100t,0.000000,0.516166,0.365098,0.812658
c9,0.091427,0.267205,0.000000,1.000000
ggs,0.589869,0.259011,0.213997,0.828650
tl,0.565306,0.469242,0.288448,0.718220
dig,0.228766,0.000000,0.199512,0.973352
clg,1.000000,0.877307,1.000000,0.000000
imt,0.067210,0.706389,0.341383,0.415062
tsm,0.305691,1.000000,0.676137,0.641300
eg,0.552924,0.795228,0.434476,0.712572
fly,0.621001,0.371455,0.359311,0.647921


This shows how each proximity value compares across the league. 0 being the lowest value and 1 the highest, with the rest of the values scaled along this range.

For comparison's sake, I'll also gather the average values across each role. This will allow me to see how each support compares to the average

In [5]:
means = [0.0]*4
for i,role in enumerate(['top','jgl','mid','adc']):
    means[i] = (df3[role].mean() - df3[role].min()) / (df3[role].max() - df3[role].min())

Next step is to find a way to graph these statistics that will show how each player compares across their role. A good way is a radar chart, which will clearly show how each team's Support compares across the league. This example shows how Golden Guardian's Support player compares. You can see that he plays around his ADC and his Toplaner more than average, but not so much around his Midlaner and Jungler

In [6]:
n = 2

nums = df3.iloc[n].tolist()
nums.append(nums[0])
means.append(means[0])

fig = go.Figure()


fig.add_trace(go.Scatterpolar(r = means, 
                    theta = ['Top','  Jungle','Mid  ','ADC', 'Top'],
                             name = "League average"))

fig.add_trace(go.Scatterpolar(r = nums, 
                    theta = ['Top','  Jungle','Mid  ','ADC', 'Top'],
                             fill = "toself",
                             name = "%s Support" % df3.index[n].upper()))


fig.update_layout(title="%s: Support proximities" % df3.index[n].upper())
fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
    range = [0,1],
        tickfont_size = 9,
        nticks=3,
        angle=180,
        tickangle=-180
    ),
  ),
  showlegend=True
)


fig.show()

This can then be repeated for the entire league

In [7]:
for n in range(10):
    nums = df3.iloc[n].tolist()
    nums.append(nums[0])

    fig = go.Figure()


    fig.add_trace(go.Scatterpolar(r = means, 
                        theta = ['Top','  Jungle','Mid  ','ADC', 'Top'],
                                 name = "League average"))

    fig.add_trace(go.Scatterpolar(r = nums, 
                        theta = ['Top','  Jungle','Mid  ','ADC', 'Top'],
                                 fill = "toself",
                                 name = "%s Support" % df3.index[n].upper()))


    fig.update_layout(title="%s: Support proximities" % df3.index[n].upper())
    fig.update_layout(
      polar=dict(
        radialaxis=dict(
          visible=True,
        range = [0,1],
            tickfont_size = 9,
            nticks=3,
            angle=180,
            tickangle=-180
        ),
      ),
      showlegend=True
    )


    fig.show()

This can be used to see the trends in how certain teams play. While it's not a complete game changer in League of Legends analysis, it can prove useful to quickly analyse an opposing team and get an idea of how they play.